In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
df1 = pd.read_csv('class_labels.csv')
df2 = pd.read_csv('go_features.csv')
df3 = pd.read_csv('gtex_features.csv')
df4 = pd.read_csv('kegg_features.csv')
df5 = pd.read_csv('pathdipall_features.csv')
df6 = pd.read_csv('ppi_features.csv')

merged_df = pd.merge(df1, df2, on='entrezId')
merged_df = pd.merge(merged_df, df3, on='entrezId')
merged_df = pd.merge(merged_df, df4, on='entrezId')
merged_df = pd.merge(merged_df, df5, on='entrezId')
merged_df = pd.merge(merged_df, df6, on='entrezId')

merged_df.fillna(0, inplace=True)
merged_df.to_csv('merged_dataset.csv', index=False)

dataset = pd.read_csv('merged_dataset.csv')
X = dataset.iloc[:, [0] + list(range(2, dataset.shape[1]))].values
Y = dataset.iloc[:, 1].values

# Feature extraction using PCA
num_components = 16  # Adjust the number of PCA components as needed
pca = PCA(n_components=num_components)
X_transformed = pca.fit_transform(X)

# Reshape the transformed data into images (assuming each row has the same number of components)
num_rows, num_components = X_transformed.shape
num_pixels = int(np.sqrt(num_components))  # Assuming the data can be represented as square images
X_images = X_transformed.reshape(-1, num_pixels, num_pixels, 1)  # Reshape to (num_rows, num_pixels, num_pixels, 1)

# Convert labels to one-hot encoding
num_classes = len(np.unique(Y))
Y_onehot = tf.keras.utils.to_categorical(Y, num_classes)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_images, Y_onehot, test_size=0.25, random_state=0)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(num_pixels, num_pixels, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 20  # Set the number of epochs here
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# Evaluate the model on the test set
Y_pred_onehot = model.predict(X_test)
Y_pred_labels = np.argmax(Y_pred_onehot, axis=1)  # Convert one-hot predictions to class labels
Y_test_labels = np.argmax(Y_test, axis=1)  # Convert one-hot ground truth to class labels



Epoch 1/20
125/125 [==============================] - 1s 4ms/step - loss: 264.4175 - accuracy: 0.9533 - val_loss: 196.1052 - val_accuracy: 0.9707
Epoch 2/20
125/125 [==============================] - 0s 2ms/step - loss: 68.2517 - accuracy: 0.9611 - val_loss: 157.1378 - val_accuracy: 0.9707
Epoch 3/20
125/125 [==============================] - 0s 2ms/step - loss: 90.7179 - accuracy: 0.9691 - val_loss: 168.0536 - val_accuracy: 0.9707
Epoch 4/20
125/125 [==============================] - 0s 2ms/step - loss: 66.1977 - accuracy: 0.9694 - val_loss: 24.6512 - val_accuracy: 0.9707
Epoch 5/20
125/125 [==============================] - 0s 2ms/step - loss: 59.5340 - accuracy: 0.9619 - val_loss: 203.7991 - val_accuracy: 0.9707
Epoch 6/20
125/125 [==============================] - 0s 2ms/step - loss: 62.5562 - accuracy: 0.9691 - val_loss: 127.3104 - val_accuracy: 0.9707
Epoch 7/20
125/125 [==============================] - 0s 2ms/step - loss: 40.0590 - accuracy: 0.9669 - val_loss: 126.4707 - val_ac

In [2]:
accuracy = accuracy_score(Y_test_labels, Y_pred_labels)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 98.65%
